# Reinforcement Learning

Goal of Reinforcement learning - choose a `policy` that will tell what `action` to take in `state s` so that to max the expected `return`.

`Policy` - $ \pi $ takes as input state $ s $ and return asction $ \alpha $ as output based on the reward value it gets (it can be described ad all astions that agent takes)

$$ \pi(s) = a $$

`State` -  $ s $ current state (position) of the agent 


`Reward` - $ r $ the reward that agent gets for being in a given state 

\begin{bmatrix} 
State & 1 & 2 & 4 & 4 \\
Reward & 0 & 0 & 0 & 100 \\
\end{bmatrix} 

`Action` - $ a $ a move (to the right in this example)

$$ -> $$

`Return` - $ R $ the final reward thet model gets after taking actions

$$ R_1 + R_2 + R_3 + R_4 = R $$

$$ 0 + 0 + 0 + 100 = 100 $$

`Discount factor`: - $ \gamma $ a factor by which we multipy reward (the reward diminished by the amount of moves)

$$ \gamma =0.9 $$

$$ 0 + 0 \cdot 0.9 + 0 \cdot (0.9)^{2} + 100 \cdot (0.9)^{3} = 72.9 $$

`MDP` - Markov Decision Process - the future only depends on the current state (past is not important)

#### State action value function

$$ Q(s,\alpha) $$

The aim is to maximize $ Q $

$$ max Q(s,\alpha) $$

Having computed $ Q(s,a) $ for every state and action, then we can compute optimal policy $ \pi(s) $

In [7]:
import numpy as np

In [152]:
states=6
actions=2

In [153]:
terminal_left_rewads=100
terminal_right_rewards=40
each_step_reward=0
gamma=0.5

misstep_prob=0

In [154]:
def get_rewards(num_states,step_reward,terminal_left_reward,terminal_right_reward):
    rewards=[step_reward]*num_states
    rewards[0]=terminal_left_reward
    rewards[-1]=terminal_right_reward
    return rewards

In [155]:
def get_transition_prob(num_states,num_actions,misstep_prob=0):
    # p[][0][] describes left moves probability
    # p[][1][] describes right moves probability
    p=np.zeros((num_states,num_actions,num_states))
    for i in range(num_states):
        if i !=0:
            p[i,0,i-1]=1-misstep_prob
            p[i,1,i-1]=misstep_prob
        if i != num_states-1:
            p[i,1,i+1]=1-misstep_prob
            p[i,0,i+1]=misstep_prob            
    p[0]=np.zeros((num_actions,num_states))
    p[-1]=np.zeros((num_actions,num_states))
    return p

In [156]:
def q_value(num_states,rewards,transition_prob,gamma,V_states,state,action):
    q_sa=rewards[state]+gamma*sum([transition_prob[state,action,sp]*V_states[sp] for sp in range(num_states)])
    return q_sa

In [157]:
def evaluate_policy(num_states,rewards,transition_prob,gamma,policy):
    max_policy_eval=10000
    treshold=1e-10
    V=np.zeros(num_states)
    for i in range(max_policy_eval):
        delta=0
        for s in range(num_states):
            v=V[s]
            V[s]=q_value(num_states,rewards,transition_prob,gamma,V,s,policy[s])
            delta=max(delta,abs(v-V[s]))
        if delta<treshold:
            break
    return V

In [158]:
def q_values(num_states,rewards,transition_prob,gamma,optimal_policy):
    q_left=np.zeros(num_states)
    q_right=np.zeros(num_states)
    V_star=evaluate_policy(num_states,rewards,transition_prob,gamma,optimal_policy)
    for s in range(num_states):
        q_left[s]=q_value(num_states,rewards,transition_prob,gamma,V_star,s,0)
        q_right[s]=q_value(num_states,rewards,transition_prob,gamma,V_star,s,1)
    return q_left,q_right

In [159]:
def improve_policy(num_states,num_actions,rewards,transition_prob,gamma,V,policy):
    policy_stable=True
    for s in range(num_states):
        q_best=V[s]
        for a in range(num_actions):
            q_sa=q_value(num_states,rewards,transition_prob,gamma,V,s,a)
            if q_sa>q_best and policy[s]!=a:
                policy[s]=a
                q_best=q_sa
                policy_stable=False
    return policy,policy_stable

In [172]:
def get_optimal_policy(num_states,num_actions,rewards,transition_prob,gamma):
    optimal_policy=np.zeros(num_states,dtype=int)
    max_policy_iter=10000
    for i in range(max_policy_iter):
        policy_stable=True
        V=evaluate_policy(num_states,rewards,transition_prob,gamma,optimal_policy)
        optimal_policy,policy_stable=improve_policy(num_states,num_actions,rewards,transition_prob,gamma,V,optimal_policy)
        if policy_stable:
            break
    return optimal_policy,V

In [179]:
rewards=get_rewards(states,each_step_reward,terminal_left_rewads,terminal_right_rewards)
rewards

[100, 0, 0, 0, 0, 40]

In [215]:
transition_prob=get_transition_prob(states,actions,misstep_prob)
transition_prob

array([[[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]],

       [[1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.]],

       [[0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.]],

       [[0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1.]],

       [[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]])

#### probabilities of moves to left side (1 mean move to this state from right state)

When agent is max left then he will stay so all probabiliriea are 0. (same for max right)

In [202]:
transition_prob[:,0,:]

array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

#### probabilities of moves to right side (1 mean move to this state from left state)

In [203]:
transition_prob[:,1,:]

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0.]])

In [205]:
optimal_policy,V=get_optimal_policy(states,actions,rewards,transition_prob,gamma)
optimal_policy

array([0, 0, 0, 0, 1, 0])

In [182]:
V

array([100. ,  50. ,  25. ,  12.5,  20. ,  40. ])

In [183]:
q_left_star,q_right_star=q_values(states,rewards,transition_prob,gamma,optimal_policy)

In [184]:
q_left_star

array([100.  ,  50.  ,  25.  ,  12.5 ,   6.25,  40.  ])

In [185]:
q_right_star

array([100.  ,  12.5 ,   6.25,  10.  ,  20.  ,  40.  ])

***

## Bellman equation

$$ Q(s,a) = R(s) + \gamma \cdot max Q(s^{'},a^{'}) $$

$$ Q(1,->) = R_1 + \gamma \cdot R_2 + \gamma{2} \cdot R_3 + \gamma{3} \cdot R_4 = R_1 + \gamma \cdot (R_2 + \gamma \cdot R_3 + \gamma{2} \cdot R_4) $$

`Q` is the `return` if you start from `state s` and take `action a (once)` and then behave `optimally after that`.

In [216]:
rewards=get_rewards(states,each_step_reward,terminal_left_rewads,terminal_right_rewards)
rewards

[100, 0, 0, 0, 0, 40]

In [217]:
transition_prob=get_transition_prob(states,actions,misstep_prob=0.1)
transition_prob

array([[[0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. ]],

       [[0.9, 0. , 0.1, 0. , 0. , 0. ],
        [0.1, 0. , 0.9, 0. , 0. , 0. ]],

       [[0. , 0.9, 0. , 0.1, 0. , 0. ],
        [0. , 0.1, 0. , 0.9, 0. , 0. ]],

       [[0. , 0. , 0.9, 0. , 0.1, 0. ],
        [0. , 0. , 0.1, 0. , 0.9, 0. ]],

       [[0. , 0. , 0. , 0.9, 0. , 0.1],
        [0. , 0. , 0. , 0.1, 0. , 0.9]],

       [[0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. ]]])

In [218]:
optimal_policy,V=get_optimal_policy(states,actions,rewards,transition_prob,gamma)
optimal_policy

array([0, 0, 0, 0, 1, 0])

In [219]:
V

array([100.        ,  46.0626341 ,  21.25268193,  10.4899317 ,
        18.52449658,  40.        ])

In [224]:
q_left_star,q_right_star=q_values(states,rewards,transition_prob,gamma,optimal_policy)
print(f'left: {q_left_star}')
print(f'fight: {q_right_star}')

left: [100.          46.0626341   21.25268193  10.4899317    6.72046926
  40.        ]
fight: [100.          14.56370687   7.02360097   9.39865756  18.52449658
  40.        ]


***

## Continuous State applications

`DQN` - deep neural network used to calculate the Q function

We use `DQN` to calculate the `Q function` by applying random `actions` in given `states` and then receive the `reward R` and compute the `Q`. Then having enough examples we train DQN to compute Q function and do it `iteratively` keeping 10000 examples in memory.

$$ x=
\begin{bmatrix} 
s \\
a \\
\end{bmatrix}
$$

$$ y = R(s) + \gamma \cdot max Q(s^{'},a^{'})
$$

The input to DQN is state s and action a. The output is Q.

Or we can make the input s and have as many outputs as actions we can make (Q given action 1, etc...).

#### $ \epsilon $- greedy policy

Take such actions that maximize current $ Q(s,a) $